In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-news-category/Data_Train.xlsx
/kaggle/input/predict-news-category/Sample_submission.xlsx
/kaggle/input/predict-news-category/Data_Test.xlsx


In [2]:
!pip install transformers

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
!pip install torch

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
df=pd.read_excel('/kaggle/input/predict-news-category/Data_Train.xlsx')
df

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3.0
1,How formidable is the opposition alliance amon...,0.0
2,Most Asian currencies were trading lower today...,3.0
3,"If you want to answer any question, click on ‘...",1.0
4,"In global markets, gold prices edged up today ...",3.0
...,...,...
7623,"Karnataka has been a Congress bastion, but it ...",0.0
7624,"The film, which also features Janhvi Kapoor, w...",2.0
7625,The database has been created after bringing t...,1.0
7626,"The state, which has had an uneasy relationshi...",0.0


In [5]:
df.rename(columns={'SECTION':'labels'},inplace=True)

In [6]:
df.rename(columns={'STORY':'text'},inplace=True)

In [7]:
# import operator as op
# l1=[]
# for i in range(len(df)):
#     if((op.countOf(df['text'][i], " ")+1)>=512):
#         l1.append(i)

In [8]:
# df.drop(l1,inplace=True)

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [10]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [11]:
df['text'] = df['text'].map(preprocess)

In [12]:
df

,text,labels
0,but the most painful was the huge reversal in ...,3.0
1,how formidable is the opposition alliance amon...,0.0
2,most asian currencies were trading lower today...,3.0
3,if you want to answer any question click on an...,1.0
4,in global markets gold prices edged up today a...,3.0
...,...,...
7623,karnataka has been a congress bastion but it a...,0.0
7624,the film which also features janhvi kapoor wil...,2.0
7625,the database has been created after bringing t...,1.0
7626,the state which has had an uneasy relationship...,0.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    7628 non-null   object 
 1   labels  7628 non-null   float64
dtypes: float64(1), object(1)
memory usage: 119.3+ KB


In [14]:
df['labels']=df['labels'].astype(int)

In [15]:
!pip install datasets

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [16]:
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 7628
})

In [17]:
def tokenize_data(example):
    return tokenizer(example['text'], max_length=512)

dataset = dataset.map(tokenize_data, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [18]:
# dataset = dataset.remove_columns("text")
# dataset = dataset.remove_columns("__index_level_0__")

In [19]:
from sklearn.model_selection import train_test_split

train_testvalid = dataset.train_test_split(test_size=0.2)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 6102
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 763
    })
    valid: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 763
    })
})

In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

In [21]:
num_labels=df['labels'].nunique()

In [22]:
from transformers import RobertaForSequenceClassification
# model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=num_labels)
# model = TFRobertaModel.from_pretrained('roberta-base')
# x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [23]:
# import torch
# from transformers import LongformerModel, LongformerTokenizer

# model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
# tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

# SAMPLE_TEXT = df['Combined']
# input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

# # Attention mask values -- 0: no attention, 1: local attention, 2: global attention
# attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
# attention_mask[:, [1, 4, 21,]] = 2  # Set global attention based on the task. For example,
#                                     # classification: the <s> token
#                                     # QA: question tokens
#                                     # LM: potentially on the beginning of sentences and paragraphs
# sequence_output, pooled_output = model(input_ids, attention_mask=attention_mask)
# loss = model(input_ids, labels=input_ids, masked_lm_labels=mlm_labels)[0

In [24]:
from transformers import Trainer
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset['train'], eval_dataset=dataset['valid'],tokenizer=tokenizer,compute_metrics=compute_metrics
)

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6102
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1146
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wa

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.168800
1000,0.052000


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
/opt/conda/lib/python3.7

TrainOutput(global_step=1146, training_loss=0.099783895319461, metrics={'train_runtime': 794.191, 'train_samples_per_second': 23.05, 'train_steps_per_second': 1.443, 'total_flos': 3292025500267920.0, 'train_loss': 0.099783895319461, 'epoch': 3.0})

In [26]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 763
  Batch size = 16


{'eval_loss': 0.10849917680025101,
 'eval_accuracy': 0.981651376146789,
 'eval_runtime': 10.264,
 'eval_samples_per_second': 74.338,
 'eval_steps_per_second': 4.677,
 'epoch': 3.0}

In [27]:
df_test=pd.read_excel('/kaggle/input/predict-news-category/Data_Test.xlsx')
df_test

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...
...,...
2743,"According to researchers, fraud in the mobile ..."
2744,The iPhone XS and XS Max share the Apple A12 c...
2745,"On the photography front, the Note 5 Pro featu..."
2746,UDAY mandated that discoms bring the gap betwe...


In [28]:
df_test.rename(columns={'STORY':'text'},inplace=True)

In [29]:
df_test['text'] = df_test['text'].map(preprocess)

In [30]:
from datasets import Dataset, DatasetDict
dataset_test = Dataset.from_pandas(df_test)
dataset_test

Dataset({
    features: ['text'],
    num_rows: 2748
})

In [31]:
dataset_test = dataset_test.map(tokenize_data, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [32]:
dataset_test

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 2748
})

In [33]:
preds=trainer.predict(dataset_test)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2748
  Batch size = 16


In [34]:
preds

PredictionOutput(predictions=array([[-2.117127 ,  7.0321665, -3.0903983, -1.1548004],
       [-1.7717983, -2.1568499,  7.5718307, -2.7106254],
       [-2.1320148,  7.041157 , -3.1043499, -1.1351066],
       ...,
       [-2.314518 ,  6.9995537, -2.8198333, -1.260513 ],
       [ 6.8706107, -1.2210412, -2.8895063, -1.5338299],
       [-1.7917279,  6.9321804, -3.131057 , -1.2601744]], dtype=float32), label_ids=None, metrics={'test_runtime': 40.7652, 'test_samples_per_second': 67.41, 'test_steps_per_second': 4.219})

In [35]:
predictions = trainer.predict(dataset_test, metric_key_prefix="predict")

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2748
  Batch size = 16


In [36]:
predictions

PredictionOutput(predictions=array([[-2.117127 ,  7.0321665, -3.0903983, -1.1548004],
       [-1.7717983, -2.1568499,  7.5718307, -2.7106254],
       [-2.1320148,  7.041157 , -3.1043499, -1.1351066],
       ...,
       [-2.314518 ,  6.9995537, -2.8198333, -1.260513 ],
       [ 6.8706107, -1.2210412, -2.8895063, -1.5338299],
       [-1.7917279,  6.9321804, -3.131057 , -1.2601744]], dtype=float32), label_ids=None, metrics={'predict_runtime': 40.5857, 'predict_samples_per_second': 67.709, 'predict_steps_per_second': 4.238})

In [41]:
y_pred = np.argmax(predictions.predictions, axis=1)

In [42]:
y_pred

array([1, 2, 1, ..., 1, 0, 1])

In [43]:
np.unique(y_pred)

array([0, 1, 2, 3])

In [44]:
df_submission=pd.DataFrame(data=y_pred,columns=['SECTION'])

In [45]:
df_submission

,SECTION
0,1
1,2
2,1
3,1
4,1
...,...
2743,1
2744,1
2745,1
2746,0


In [46]:
df_submission.to_csv('Submission.csv')